In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/covid19-us/us-counties.csv')
df['date'] = pd.to_datetime(df.date)
df['date1'] = df['date'].dt.strftime("%Y%m%d").astype(int)
df2 = df

In [ ]:
df2a = df2[df2['date'] == df['date'].max()]

In [ ]:
df2['State_County'] = df2['state'].astype(str) + ">" + df2['county'].astype(str) 

In [ ]:
State_Count_list = set()
for a in df2['State_County']:
    State_Count_list.add(a)

In [ ]:
len_list = len(State_Count_list)
print(len_list)

## Difference Table for Each County

In [ ]:
import time
df3 = pd.DataFrame()
count = 0
s = time.time()

for aa in State_Count_list:

    dfa = df2[df2['State_County'] == aa][['fips','cases','deaths']].diff() 
    dfa["state_county"] = aa
    dfa['date'] = df2[df2['State_County'] == aa]['date']
    df3 = pd.concat([df3,dfa])
    
    count+=1
    if count % 500 == 0:
        e = time.time()
        print("Time: {} | Percent Done: {}: ".format((e-s)/60,(count/len_list)))

In [ ]:
df3.tail()

## Rolling Window (3day sum) Table for Each County

In [ ]:
df3a = pd.DataFrame()

count = 0
s = time.time()

for aa in State_Count_list:
    df = df3[df3['state_county'] == aa]
    dfa = df[['cases','deaths']].rolling(window=3).sum()
    dfa = dfa.rename(columns= {'cases': "Cases_Sum3", 'deaths': "Deaths_Sum3"})
    dfb = df[['cases','deaths']].rolling(window=3).mean()
    dfb = dfb.rename(columns= {'cases': "Cases_Mean3",'deaths':"Deaths_Mean3"})
    
    dfc = df[['cases','deaths']].rolling(window=6).sum()
    dfc = dfc.rename(columns= {'cases': "Cases_Sum6", 'deaths': "Deaths_Sum6"})
    dfd = df[['cases','deaths']].rolling(window=6).mean()
    dfd = dfd.rename(columns= {'cases': "Cases_Mean6",'deaths':"Deaths_Mean6"})
    
    dfaa = pd.concat([dfa,dfb,dfc,dfd], axis = 1)
    dfaa[['state_county','date']] = df3[df3['state_county'] == aa][['state_county','date']]
    
    df3a = pd.concat([df3a,dfaa])
    count +=1
    if count % 50 == 0:
        e = time.time()
        print("Time: {} Mins | Percent Done: {}: ".format(((e-s)/60),(count/len_list)))
print('Done | Time: {} Minutes'.format((e-s)/60))

In [ ]:
df3a

In [ ]:
df4 = pd.DataFrame()
df3aa = df3a[(df3a['date'] > '2021-07-20')]
for a in State_Count_list:
    if a in df3aa['state_county'].unique():
        df = df3aa[df3aa['state_county'] == a]
        
        df1 = df.mean(numeric_only = True)
        df1['state_county'] = a
        df1 = df1.to_frame().T
        df1 = pd.DataFrame(df1)
        df4 = pd.concat([df4,df1])

In [ ]:
df4.sort_values('Cases_Mean3',ascending = False).head(20)

In [ ]:
df4[['state_county','Cases_Mean3']].set_index('state_county').sort_values('Cases_Mean3',ascending = False).head(20).plot(kind = 'barh',figsize = (15,10), title = '20 Counties with Highest Cases last 3 days')

df4[['state_county','Cases_Mean6']].set_index('state_county').sort_values('Cases_Mean6',ascending = False).head(20).plot(kind = 'barh',figsize = (15,10), title = '20 Counties with Highest Cases last 6 days')


In [ ]:
df4[['state_county','Deaths_Mean3']].set_index('state_county').sort_values('Deaths_Mean3',ascending = False).head(20).plot(kind = 'barh',figsize = (15,10), title = '20 Counties with Highest Deaths last 3 days')

df4[['state_county','Deaths_Mean6']].set_index('state_county').sort_values('Deaths_Mean6',ascending = False).head(20).plot(kind = 'barh',figsize = (15,10), title = '20 Counties with Highest Deaths last 6 days')


In [ ]:
df3.to_csv("Diff.csv")

In [ ]:
df3a.to_csv("Wind_3.csv")